<a href="https://colab.research.google.com/github/Segtanof/pyfin/blob/main/07_Case_Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Case Study

The idea is that you will work on a case study to deepen (and expand!) your knowledge of Python, WRDS databases and all other concepts we have learned in the course up to this point.

Unless the task explicitly says "manually" do something, you should solve all questions with code.

The case study is similar to the take-home exam.

## Imports

Please collect all imports in the cells below and do not import more packages further down in the notebook.

In [ ]:
!pip install pingouin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 7.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import pingouin as pg

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Introductory exercises

In the following, you have a list of tickers.

- Manually comment out the ticker with length 3.
- How many tickers are in the list?
- Collect the second-to-last character of all tickers in a list.
- Collect all tickers in a list that contain the letter "i" (case insensitive).
- How many letters "S" are in all of the tickers together?
- Make a list that has tuples as elements. The first element in each tuple should be the ticker. The second element in each tuple should be the index position of the ticker in the list, squared.

In [3]:
list_of_tickers = [
    'ISDS',
    'ISDX',
    'FLSA',
    'GTiP',
    'BSDE',
    'XKCP',
#    'SWI',
]


In [4]:
len(list_of_tickers)

6

In [5]:
second = [ticker[-2] for ticker in list_of_tickers]
second 

['D', 'D', 'S', 'i', 'D', 'C']

In [6]:
ilist = [ticker for ticker in list_of_tickers if "i" in ticker.lower()]

# ilist = []
# for ticker in list_of_tickers:
#     if "i" in ticker:
#         ilist.append(ticker)

ilist

['ISDS', 'ISDX', 'GTiP']

In [7]:
slist = []
for ticker in list_of_tickers:
    for i in ticker:
        if i == "S":
            slist.append(i)

len(slist)


5

## Downloading and preparing data

Download CRSP data from WRDS. Download daily stock price data for the period from 2019-01-01 to 2019-12-31 for all companies.

Select the following columns:
- permco
- ticker
- siccd
- prc
- ret
- shrout
- vwretd

Note that permno (a share class identifier) will be downloaded automatically.

- Upload it to your Google Drive.
- Load it into Colab.
- Clean the column names by converting them to lowercase.

**IF YOU DO NOT (YET) HAVE ACCESS TO WRDS**

Uncomment the lines below to load provided data.

- [Variable descriptions](https://drive.google.com/file/d/1TRNV8KZBMk6bwHFJgDxtFdUHMPHLYerE/view?usp=sharing)
- [shrout details](https://drive.google.com/file/d/1puGQXoAnRdu3rfpYg3KPWZXuLGUokARB/view?usp=sharing)

In [ ]:
crsp_file = "/content/drive/MyDrive/Colab Notebooks/Python course 2024-11/data/crsp_casestudy.csv.gz"

# If you do not have access to WRDS yet, uncomment the following lines
# !gdown 163Xu08XdGnFeMxCf7yOeirm3Uj_Mx2W5
# crsp_file = "/content/crsp_casestudy.csv.gz"

Downloading...
From: https://drive.google.com/uc?id=163Xu08XdGnFeMxCf7yOeirm3Uj_Mx2W5
To: /content/crsp_casestudy.csv.gz
100% 26.2M/26.2M [00:00<00:00, 55.2MB/s]


Ensure we have numeric datatypes for the relevant variables.

Show the first 3 rows

In [8]:
data = pd.read_csv("casestudydata.zip",parse_dates=["date"])
data.columns = data.columns.str.lower()
data["ret"]= pd.to_numeric(data["ret"], errors="coerce")
data.info()


C:\Users\godfr\AppData\Local\Temp\ipykernel_19264\1935311578.py:1: DtypeWarning: Columns (2,6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("casestudydata.zip",parse_dates=["date"])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1912001 entries, 0 to 1912000
Data columns (total 9 columns):
 #   Column  Dtype         
---  ------  -----         
 0   permno  int64         
 1   date    datetime64[ns]
 2   siccd   object        
 3   ticker  object        
 4   permco  int64         
 5   prc     float64       
 6   ret     float64       
 7   shrout  float64       
 8   vwretd  float64       
dtypes: datetime64[ns](1), float64(4), int64(2), object(2)
memory usage: 131.3+ MB


## Adding market capitalization and industry

### Industry

Add the 2-digit SIC code industry as a column to your data.

A 2-digit SIC code industry means that the first 2 numbers in the SIC code are considered to be the same industry.

Note that these should be strings!

In [9]:
data["industry"] = [x[:2] for x in data["siccd"].astype(str)]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1912001 entries, 0 to 1912000
Data columns (total 10 columns):
 #   Column    Dtype         
---  ------    -----         
 0   permno    int64         
 1   date      datetime64[ns]
 2   siccd     object        
 3   ticker    object        
 4   permco    int64         
 5   prc       float64       
 6   ret       float64       
 7   shrout    float64       
 8   vwretd    float64       
 9   industry  object        
dtypes: datetime64[ns](1), float64(4), int64(2), object(3)
memory usage: 145.9+ MB


### Market capitalization

Calculate the market capitalization `mcap` from `prc` and `shrout` and add it to the DataFrame as a new column.

Check the variable description of `shrout` to avoid mistakes.

## Getting an overview

How many unique companies do we have? Print it in a nice format with a comma `,` as the thousands separator.

How many share classes `permno` does each company have on average? Print it in a nice format rounded to two decimal places.

Plot a bar chart that shows how many companies are in each 2-digit SIC code industry. Plot only industries with more than 50 companies. Sort the values, so that the industry with the most companies is on the left.

## Price plot

Consider data as of 2019-05-16. Get the companies that satisfy the following conditions:
- Market cap greater or equal to USD 2.5 billion
- Not in the industry "99"
- Return greater than 0.05

How many are there?

**For the rest of the case study, we will only work with these companies.**

If you can't solve it, pick 10 companies however you like and work with them going forward.

Plot these companies' share price development over the entire year.

## Analyzing the data

Now `pivot` (hint!) the (filtered) data to get the `ret` as values, the `date` as index and the `ticker` in the columns.

Calculate the return correlations of the companies.

Since that's hard to grasp quickly, plot a heatmap. Choose a diverging colormap from the [available ones](https://seaborn.pydata.org/tutorial/color_palettes.html).

Which company pair has the highest correlation? Get it via code. Ignore correlations of the companies with themselves. [Difficult task!]

## Portfolio weighting schemes

In many scenarios, we want to build portfolios of companies. When combining them, a choice has to be made on how to weight them.

The easiest way is equal weighting. A very popular way is value weighting.

In the following, we will want to convert from returns to prices in order to plot meaningful stock price developments. Write a function that takes a return column as input and returns a column of prices as output.

The function should have a keyword argument `initial_value` that is set to `100` by default and is the amount of money invested in the first period.

In [ ]:
# Test your function with these returns
test_returns = pd.Series([0.1, -0.1, 0.2, -0.2])

### Equal weighting

Equal weighting means that all stocks in a portfolio have the same weight.

Calculate an equal weighted portfolio return and plot it over the year.

### Value weighting

Value weighting means that stocks are weighted proportionally to their market capitalization. Thus, we need to calculate their percentage of the total market cap at each point in time. This does not require a loop!

Hint: We can split the problem into two parts. A returns DataFrame and a weights DataFrame, which we can then multiply with each other.



*Hint: Get a market capitalization DataFrame similar to the returns data via `pivot`.*

Plot a stacked barchart that shows the average proportion invested in each company in each month.

Check the documentation for [pd.Grouper](https://pandas.pydata.org/docs/reference/api/pandas.Grouper.html), which is used for advanced grouping. It is particularly useful for grouping based on time frequencies.

Plot the performance of the value weighted portfolio over the year.

### Comparison

It's hard to tell the difference between the performance and the equal-weighted portfolio and the value-weighted porfolio, so let's plot both lines in one chart and label them accordingly. Hint: Use `pd.concat` for this.

Which one performs better?

## Statistical tests

Extract the values from the `vwretd` column of the data and merge them with the equal-weighted and value-weighted returns into one dataframe that has 3 columns, `['ewret','vwret','vwretd']`.
Look at WRDS to find out what the `vwretd` column means.

We now have the returns of 3 different portfolios in one dataframe.

What is the mean return for each portfolio?

Run pairwise ttests for equality of means between all column combinations. Which portfolio combination is the most likely to have similar means (check manually).

*Hint: Check the documentation, there is a shortcut.*

Check whether the return correlation between the different portfolios is statistically significant.